In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import collections

from tf.app import use

In [3]:
A = use('oldbabylonian', hoist=globals(), check=True)

TF app is up-to-date.
Using annotation/app-oldbabylonian commit b857e6fd557083e3f77f3055d5e74b6245e84fdd (=latest)
  in /Users/johnsojc/text-fabric-data/__apps__/oldbabylonian.
No new data release available online.
Using Nino-cunei/oldbabylonian/tf - 1.0.4 rv1.4 (=latest) in /Users/johnsojc/text-fabric-data.


# Prepositions

Prepositions play a role in finding nouns.

Here we do some basic exploration.

This is a list of prepositions:

```
i-na
a-na
e-li
isz-tu
it-ti
```

# Exploratory excursions

Before we formulate a query to draw out the prepositions and their subsequent words,
let's do some exploration. 
We'll need queries that are a bit more advanced as well, and we'll take a moment to explain
what is going on.

The first question is: do prepositions stand on their own as words, or are they part of words?
Can they cross word boundaries?

Let's check this out for the first preposition `i-na`:

## i-na as complete word

First we look for all complete words that are made up of two signs, with readings `i` and `na` respectively.

In [4]:
queryW = '''
word
  =: sign reading=i
  <: sign reading=na
  :=
'''

Remember: 

* indentation in the template above means means that the indented objects 
  must be contained (embedded) in its less indented parent object:
  the signs are embedded in the word;
* `=:` connects the `words` with the first `sign`,
  and the meaning is: both must *start* at the same slot;
* `<:` connects the first sign with the second one,
  and the meaning is that the first sign must immediately precede the second one;
* `:=` connects the second sign with the word,
  and the meaning is that both must *end* at the same slot.
  
**Hint** Do you find it hard to remember what is what between `:=` and `=:`?
Look at the position of the `=`:

* in `:=` the `=` comes last, and the meaning is:
  both *end* at the same slot.
* in `=:` the `=` comes first, and the meaning is:
  both *start* at the same slot.

In [5]:
resultsW = A.search(queryW)

  0.60s 1440 results


In [6]:
A.table(resultsW, end=100)

n,p,word,sign,sign
1,P509373 obverse:12,i-na,i-,na
2,P509373 reverse:11',i-[na],i-,[na]
3,P509374 obverse:15,i-na,i-,na
4,P509375 reverse:9,i-na,i-,na
5,P509376 reverse:4,i-na,i-,na
6,P509377 obverse:1,i-na,i-,na
7,P509377 reverse:4,i-na,i-,na
8,P509377 reverse:5,i-na,i-,na
9,P509377 reverse:12,i-na,i-,na
10,P481192 reverse:1',i-na,i-,na


These are the first 100 occurrences of the preposition i-na "in, at" and if you click on the highlighted p-number, you will be taken to the CDLI page for that document. If you click on P510606 reverse:6 in no. 99, you'll be taken to the the corresponding CDLI page, otherwise accessible at [http://cdli.ucla.edu/P510606]. There, on line 6 of the reverse, we find: 6. hi-sze#-eh-tam ma-la i-na ka#-\[ap-ri-im\].

Since most prepositions are followed by a syllabically written noun in the genitive case, the different prepositions provide us with an easy way of identifying a large group of nouns in a particular case. So let's look at the words that immediately follow the preposition ina and see what kinds of words show up in that position.

In [7]:
queryL = '''
line
  word
    =: sign reading=i
    <: sign reading=na
    :=
  <: word
'''
results = A.search(queryL)
A.table(results, end=30)

  0.61s 1433 results


n,p,line,word,sign,sign,word
1,P509373 obverse:12,u3 i-na _uru_ x-szum{ki} sza-ak-nu id-di-a-am-ma,i-na,i-,na,_uru_
2,P509373 reverse:11',a-na ki-ma i-[na] _dub e2-gal_-lim,i-[na],i-,[na],_dub
3,P509374 obverse:15,i-na _ka2 ki-lam_ sza-mi-im-ma,i-na,i-,na,_ka2
4,P509375 reverse:9,i-na la-hi-a-nim,i-na,i-,na,la-hi-a-nim
5,P509376 reverse:4,i-na tup-pi2-ka pa-nam szu-ur2-szi-a-am-ma,i-na,i-,na,tup-pi2-ka
6,P509377 obverse:1,i-nu-ma i-na _e2_ da-[...],i-na,i-,na,_e2_
7,P509377 reverse:4,1(disz) _amar mu_ 3(disz) i-na i-ni-ia ta#-[...],i-na,i-,na,i-ni-ia
8,P509377 reverse:5,i-na _gu4-hi-a_-ia ta-as-su2-uq-ma a-na# [...],i-na,i-,na,_gu4-hi-a_-ia
9,P509377 reverse:12,i-na la _sza3-gal_ la i-mu#-[tu],i-na,i-,na,la
10,P481192 reverse:1',i-na pu#-uh2#-ri i-na ma-har# a-wi#-[le-e],i-na,i-,na,pu#-uh2#-ri


These are only the first 30 items out of the more than 1400 currently in the corpus. In fact, in these first 30 occurrences there are only a couple that exhibit a syllabically written noun in the genitive singular such as i-na pu-uh2-ri-im in item numbers 14 or 15. The entire line 6 in P510527, which corresponds to AbB 12, 2, reads as follows: 6. {disz}ip-qu2-i3-li2-szu \_di-ku5\_ i-na pu-uh2-ri-im. AbB 12 is an abbreviation for W. H. van Soldt's Letters in the British Museum, published in 1990. Reasonably enough, van Soldt translates the line as follows: "Ipqu-ilišu the judge (has spoken at length against me) in the assembly," where i-na pu-uh2-ri-im corresponds to "in the assembly".

Besides syllabically written nouns, there are many other typse of nouns that follow i-na in this small subset: (i) logograms such as URU, DUB, E2 or GU4.HI.A, (ii) numerals such as 4(disz) or 6(disz), (iii) following nouns in construct, which do not explictly show the genitive case, such as qa2-ti in P510530 and (iv) last but not least, a number of examples in which i-na combines with an infinitive to form a temporal clause.

# WORK IN PROGRESS

In [8]:
queryL = '''
line
  word
    =: sign reading=isz
    <: sign reading=tu
    :=
  <: word
'''
results = A.search(queryL)
A.table(results, end=30)

  0.58s 251 results


n,p,line,word,sign,sign,word
1,P509373 obverse:11,sza _{d}utu_-ha-zi-[ir] isz-tu _mu 7(disz) kam_ id-di-nu-szum,isz-tu,isz-,tu,_mu
2,P509376 obverse:1,isz-tu _iti sig4-a_,isz-tu,isz-,tu,_iti
3,P510535 obverse:11,isz-tu i-na-an-na _u4 2(disz)-kam_,isz-tu,isz-,tu,i-na-an-na
4,P510542 reverse:5,isz#-tu u2-ur# _e2 i3-dub_,isz#-tu,isz#-,tu,u2-ur#
5,P510543 obverse:11,[am-mi]-nim# isz-tu ta-al-li-<<li>>-ku,isz-tu,isz-,tu,ta-al-li-<<li>>-ku
6,P510556 obverse:10,isz-tu du#-ri-im ma-ti-ma sze-am# i-na pa-ni-ia#,isz-tu,isz-,tu,du#-ri-im
7,P510556 obverse:15,isz-tu sze-um a-na ma-asz-ka-nim i-lu-u2,isz-tu,isz-,tu,sze-um
8,P510562 reverse:1,i-na#-an-na isz-tu _sza3_ ma-tim _iti 1(disz)-kam_,isz-tu,isz-,tu,_sza3_
9,P510572 reverse:9,[isz]-tu# _mu 2(disz)-kam_ aq-bi-kum,[isz]-tu#,[isz]-,tu#,_mu
10,P510583 obverse:14,isz-tu ma-at,isz-tu,isz-,tu,ma-at


In [9]:
queryL = '''
line
  word
    =: sign reading=it
    <: sign reading=ti
    :=
  <: word
'''
results = A.search(queryL)
A.table(results, end=30)

  0.59s 141 results


n,p,line,word,sign,sign,word
1,P509375 reverse:2,it-ti _dam-gar3_,it-ti,it-,ti,_dam-gar3_
2,P509376 reverse:10,it-ti na-bi-{d}suen a-na _babila2{ki}_,it-ti,it-,ti,na-bi-{d}suen
3,P510532 reverse:8,it#-ti# ip#-qu2-i3-li2-szu szu-a-ti,it#-ti#,it#-,ti#,ip#-qu2-i3-li2-szu
4,P510571 reverse:9,it-ti ib-ri-ka,it-ti,it-,ti,ib-ri-ka
5,P510579 obverse:9,it-ti i-la-tim,it-ti,it-,ti,i-la-tim
6,P510580 obverse:11,it-ti su-mu-ha-am-mu,it-ti,it-,ti,su-mu-ha-am-mu
7,P510583 reverse:19,"t,u2-ur-da-asz-szi it-ti {d}suen-i-din-nam",it-ti,it-,ti,{d}suen-i-din-nam
8,P510584 reverse:18,it-ti {d}suen-i-din-nam#,it-ti,it-,ti,{d}suen-i-din-nam#
9,P510585 obverse:11,it#-ti {d}marduk-mu-sza-lim _dub#-sar_ [(x)],it#-ti,it#-,ti,{d}marduk-mu-sza-lim
10,P510589 reverse:13,it#-[ti] _ugula# e2_ i-na zimbir{ki}-am-na-num a-na-ku,it#-[ti],it#-,[ti],_ugula#


Just in case you wonder whether a word can cross a line boundary, let us check that.

In [10]:
suspect = []

for w in F.otype.s('word'):
  signs = L.d(w, otype='sign')
  firstSign = signs[0]
  lastSign = signs[-1]
  firstLine = L.u(firstSign, otype='line')[0]
  lastLine = L.u(lastSign, otype='line')[0]
  if firstLine != lastLine:
    suspect.append((w, firstLine, lastLine))
    
len(suspect)

0

Rest reassured: every word lies neatly in its own single line.

## `i` followed by `na` anywhere

Now we look for adjacent pairs of signs of which the first has reading `i` and the second reading `na`.
We do not care whether both signs are in the same word or even the same line, or face or document!

In [11]:
query = '''
sign reading=i
<: sign reading=na
'''

In [12]:
results = A.search(query)

  0.50s 1768 results


Definitely more results. We want to show the extra cases.

There are two possibilities:

* the signs lie within a bigger word
* the signs lie in two distinct words

Let's query for them separately and see whether the numbers add up.

In [13]:
queryB = '''
word
/with/
  sign
  <: sign
  <: sign
/-/
  sign reading=i
  <: sign reading=na
'''

Explanation: in this query you see a new construct: a *quantifier*, namely `/with/` ... `/-/`.

The *with* imposes a condition on the preceding atom: `word`.
This particular condition states that the word must have at least three consecutive signs.
These three signs will not be listed in the results.

Words that satisfy this condition, will then be matched against the rest: that there are adjacent signs with `i` and `ma`.

For more quantifiers, read the
[docs](https://annotation.github.io/text-fabric/Use/Search/#quantifiers)

In [14]:
resultsB = A.search(queryB)

  1.79s 325 results


In [15]:
A.table(resultsB, end=10)

n,p,word,sign,sign
1,P509376 obverse:4,i-na-an-na,i-,na-
2,P509376 obverse:9,i-na-an-na,i-,na-
3,P510530 obverse:14,i-na-an-na,i-,na-
4,P510534 reverse:11',i-na-an-na,i-,na-
5,P510535 obverse:11,i-na-an-na,i-,na-
6,P510535 reverse:4,i-na-ad-di-na-ak-kum,i-,na-
7,P510536 reverse:1,i-na-an-na,i-,na-
8,P510541 reverse:7,i-na#-an#-na#,i-,na#-
9,P510544 reverse:1,i-na#-an#-na#,i-,na#-
10,P510546 obverse:11,i-na-ad-di-na-ak#-[kum],i-,na-


It seems that these words happen to start with `i-na`. Is that the case for all of these?

Let's filter out the ones that do not start with `i`.

In [16]:
nonIstart = [(w, s1, s2) for (w, s1, s2) in resultsB if F.reading.v(s1) != 'i']
len(nonIstart)

0

So, indeed, all these cases are words that start with `i-na`, so we can consider these as words with the preposition `i-na` prefixed.

Well, 1440 + 325 = 1765, so we expect 3 cases where the `i` and `na` split a word.

In [17]:
queryS = '''
w1:word
  s1:sign reading=i

w2:word
  s2:sign reading=na
  
w1 < w2
s1 <: s2
'''

Explanation. Again something new: we can give *names* to the objects in our query and use those names
to state relational constraints later on.

So here we look for two words, one containing a sign `i` and the other containing a sign `na`.

The extra conditions are that `w1` and `w2` are not equal (in fact, we state that `w1` comes before `w2`.

And the two signs must be adjacent: `s1 <: s2`.

In [18]:
resultsS = A.search(queryS)

  0.54s 3 results


In [19]:
A.table(resultsS)

n,p,word,sign,word,sign
1,P313316 obverse:9,ni-qi2-i,i,na-mu-[x,na-
2,P313347 obverse:10,i,i,na,na
3,P387297 obverse:6,pi-i,i,na-ri-i-im,na-


For context, we query them in their lines:

In [20]:
queryS = '''
line
  w1:word
    s1:sign reading=i

  w2:word
    s2:sign reading=na
  
w1 < w2
s1 <: s2
'''

In [21]:
resultsS = A.search(queryS)

  0.57s 3 results


In [22]:
A.table(resultsS)

n,p,line,word,sign,word,sign
1,P313316 obverse:9,i-na ni-qi2-i na-mu-[x x] im,ni-qi2-i,i,na-mu-[x,na-
2,P313347 obverse:10,u3 a-di i na x x gu-na-ak-kum,i,i,na,na
3,P387297 obverse:6,_5(gesz2) gur_ a-na pi-i na-ri-i-im,pi-i,i,na-ri-i-im,na-


Finally, we come up with a query that looks for all `i-na` that are at the start of a word, including the cases where 
`i-na` is the whole word. In that case, we also want the next word included in our query.

We cannot easily express this kind of if statement within our query, so we just ask for the words starting with `i-na`
and we'll process the results to get the following word, if needed.

If there is an `i-na` at the end of a line, we are not interested in it.

In [23]:
query = '''
line
  word
    =: sign reading=i
    <: sign reading=na
'''

In [24]:
results = A.search(query)

  0.61s 1760 results


In [25]:
A.table(results, end=10)

n,p,line,word,sign,sign
1,P509373 obverse:12,u3 i-na _uru_ x-szum{ki} sza-ak-nu id-di-a-am-ma,i-na,i-,na
2,P509373 reverse:11',a-na ki-ma i-[na] _dub e2-gal_-lim,i-[na],i-,[na]
3,P509374 obverse:15,i-na _ka2 ki-lam_ sza-mi-im-ma,i-na,i-,na
4,P509375 reverse:9,i-na la-hi-a-nim,i-na,i-,na
5,P509376 obverse:4,_u4 4(disz)-kam_ a-di i-na-an-na,i-na-an-na,i-,na-
6,P509376 obverse:9,i-na-an-na na-bi-{d}suen,i-na-an-na,i-,na-
7,P509376 reverse:4,i-na tup-pi2-ka pa-nam szu-ur2-szi-a-am-ma,i-na,i-,na
8,P509377 obverse:1,i-nu-ma i-na _e2_ da-[...],i-na,i-,na
9,P509377 reverse:4,1(disz) _amar mu_ 3(disz) i-na i-ni-ia ta#-[...],i-na,i-,na
10,P509377 reverse:5,i-na _gu4-hi-a_-ia ta-as-su2-uq-ma a-na# [...],i-na,i-,na


# All propositions

Now let's query for all words that start with a proposition.

In [26]:
query = '''
word
/with/
  =: sign reading=i
  <: sign reading=na
/or/
  =: sign reading=a
  <: sign reading=na
/or/
  =: sign reading=e
  <: sign reading=li
/or/
  =: sign reading=isz
  <: sign reading=tu
/or/
  =: sign reading=it
  <: sign reading=ti
/-/
'''

Explanation: we have already encountered the `/with/` quantifiers. Now we see it in its full form: as a list of alternatives.

Again: none of the sign nodes that attest the alternatives, end up in the results.

That does not matter, we only want the words.

With the list of words in hand, we are going to make a hand-coded tuple of all cases with nice highlighting.

In [27]:
rawResults = A.search(query)

  0.56s 6840 results


In [28]:
rawResults[0:10]

[(258163,),
 (258171,),
 (258199,),
 (258205,),
 (258261,),
 (258274,),
 (258276,),
 (258314,),
 (258336,),
 (258362,)]

`rawResults` is a tuple of singleton tuples, each containing just a word node.

In the following code, we walk through all the results, and

* retrieve the containing line
* highlight the preposition within the word
* highlight the remaining material in the word in a different color
* if there is no remaining material in the word, draw in the next word and highlight that.

In [29]:
results = []
highlights = {}

for (w,) in rawResults:
  line = L.u(w, otype='line')[0]
  signs = L.d(w, otype='sign')
  highlights[signs[0]] = 'orange'
  highlights[signs[1]] = 'orange'
  if len(signs) > 2:
    noun = signs[2:]
  else:
    nextWord = L.n(w, otype='word')[0]
    noun = L.d(nextWord, otype='sign')
  for s in noun:
    highlights[s] = 'aquamarine'
  results.append((line, w, nextWord))

In [30]:
A.table(results, end=10, highlights=highlights)

n,p,line,word,word
1,P509373 obverse:1,[a-na] _{d}suen_-i-[din-nam],[a-na],_{d}suen_-i-[din-nam]
2,P509373 obverse:4,_{d}utu_ u3 _{d}[marduk]_ a-na da-ri-a-[tim],a-na,da-ri-a-[tim]
3,P509373 obverse:11,sza _{d}utu_-ha-zi-[ir] isz-tu _mu 7(disz) kam_ id-di-nu-szum,isz-tu,_mu
4,P509373 obverse:12,u3 i-na _uru_ x-szum{ki} sza-ak-nu id-di-a-am-ma,i-na,_uru_
5,P509373 reverse:8',a-[na] sa-[am-su]-ba-ah-[la] x x li ig bu,a-[na],sa-[am-su]-ba-ah-[la]
6,P509373 reverse:11',a-na ki-ma i-[na] _dub e2-gal_-lim,a-na,ki-ma
7,P509373 reverse:11',a-na ki-ma i-[na] _dub e2-gal_-lim,i-[na],_dub
8,P509374 obverse:1,a-[na] i3-li2-i-qi2-sza-am,a-[na],i3-li2-i-qi2-sza-am
9,P509374 obverse:7,_dub_-pi2 a-na nu-un-na-tum,a-na,nu-un-na-tum
10,P509374 obverse:15,i-na _ka2 ki-lam_ sza-mi-im-ma,i-na,_ka2


In [31]:
A.table(results, start=1000, end=1010, highlights=highlights)

n,p,line,word,word
1000,P510698 obverse:11,szum-ma i-na ki-tim a-bi,i-na,ki-tim
1001,P510698 obverse:13,"i-na an-ni-tim et,-ra-an-ni-i-ma",i-na,an-ni-tim
1002,P510698 obverse:14,lu wa-ra-ad-ka sza da-ri#-a-tim a-na#-ku#,a-na#-ku#,an-ni-tim
1003,P510699 obverse:1,a-na i3-li2-x-x-AD,a-na,i3-li2-x-x-AD
1004,P510699 obverse:4,a-na mi-ni-im,a-na,mi-ni-im
1005,P510699 obverse:5,a-na tu-uk-la-ti,a-na,tu-uk-la-ti
1006,P510699 reverse:5,a-na szu-mi-a-hi-ia,a-na,szu-mi-a-hi-ia
1007,P510699 reverse:8,a-na-ku _ku3-babbar_,a-na-ku,szu-mi-a-hi-ia
1008,P510700 obverse:1,a-na sza-pi2-ri-ia!(I) qi2-bi2-ma,a-na,sza-pi2-ri-ia!(I)
1009,P510700 obverse:5,i-na {iri}dag-la-a _gu4 hi-a_-ni ka-su2-ma,i-na,{iri}dag-la-a


In [32]:
A.table(results, start=6000, end=6010, highlights=highlights)

n,p,line,word,word
6000,P373015 obverse:12,ki-ma i-na u2-ri-im-ma,i-na,u2-ri-im-ma
6001,P373015 left:1,sza a-wa-ti-sza a-na ga-gi-im ga-am-ra-ti#,a-na,ga-gi-im
6002,P373016 obverse:1,a-[na ...],a-[na,...]
6003,P373016 obverse:9,isz#-tu _<iti> sze-sag11-ku5 u4 1(disz)-kam_,isz#-tu,_<iti>
6004,P373016 reverse:1,a-na _iri_ kisz{ki},a-na,_iri_
6005,P373016 reverse:4,a-na babila2{ki},a-na,babila2{ki}
6006,P373016 left:2,[u3] ARAD2-{d}nin-szubur a-na _u4 2(disz)-kam_ i-sa-ni-<qa2>-ki,a-na,_u4
6007,P373017 obverse:1,a-na {d}suen-ma-[gir],a-na,{d}suen-ma-[gir]
6008,P373017 obverse:8,a-na _szuku erin2_ a-na lu2-{d}asal-lu2-hi,a-na,_szuku
6009,P373017 obverse:8,a-na _szuku erin2_ a-na lu2-{d}asal-lu2-hi,a-na,lu2-{d}asal-lu2-hi


Finally, lets export this table to Excel.

We use the function [`A.export()`](https://annotation.github.io/text-fabric/Api/App/#display).

If we pass it our results and nothing else, it will write them to `results.tsv`, a file you
can open in Excel.

We will not see the highlights there, though.

In [33]:
A.export(results)

But we also like it in rich text and in unicode:

In [34]:
A.export(results, toFile='resultsRich.tsv', fmt='text-orig-rich')
A.export(results, toFile='resultsUnicode.tsv', fmt='text-orig-unicode')

Here are a few screenshots:

![resultsRich](images/prepExcelRich.png)

In the next one we have manually set the font to Santakku in Excel for the relevant columns.

![resultsUnicode](images/prepExcelUnicode.png)

Note that these Excel files have nearly 7000 rows. 

You can filter as you like. 

It is also easy to include more columns with refined information on the basis of which you can group and sort in additional ways.